In [1]:
from Trinity import TrinityClassifier
import pandas as pd

In [2]:
training_data_wt = pd.read_csv('Nata_Files/less_features/train_data_logreg.csv', index_col=0)
testing_data_wt = pd.read_csv('Nata_Files/less_features/test_data_logreg.csv', index_col=0)
prediction_data = pd.read_csv('Nata_Files/less_features/pred_data_logreg.csv', index_col=0)
pastel_d_nata_lrn = pd.read_csv('Nata_Files/learn.csv', index_col=0)
pastel_d_nata_lrn = pastel_d_nata_lrn[pastel_d_nata_lrn['quality_class'].isna() == False]

In [3]:
train_data = training_data_wt.drop('target', axis=1)
train_target = training_data_wt[['target']]
test_data = testing_data_wt.drop('target', axis=1)
test_target = testing_data_wt[['target']]

In [4]:
tc = TrinityClassifier(train_data.shape[1], 300, 0.2)

In [5]:
tc.train(X=train_data, y=train_target)

Epoch:    0 | Avg Error: 0.414200 | Learning Rate: 0.200000
Epoch:  100 | Avg Error: 0.332413 | Learning Rate: 0.121154
Epoch:  200 | Avg Error: 0.332337 | Learning Rate: 0.073392

Training completed. Final learning rate: 0.044458


In [6]:
tc.score(train_data, train_target)

0.7573264781491003

In [7]:
tc.score(test_data, test_target)

0.7630769230769231

In [11]:
tc_mom = TrinityClassifier(train_data.shape[1], 500, 0.1, 0.9)

In [ ]:
tc_mom.train(X=train_data, y=train_target)

In [ ]:
tc_mom.score(train_data, train_target), tc_mom.score(test_data, test_target)

In [8]:
predictions = tc.predict(prediction_data.drop('origin_dummy', axis=1))

In [9]:
predictions.to_csv('Nata_Files/predictionsNN.csv')

In [10]:
predictions

id
5201    OK
5202    OK
5203    OK
5204    OK
5205    OK
        ..
6496    OK
6497    OK
6498    OK
6499    OK
6500    OK
Length: 1300, dtype: object